In [1]:
import numpy as np
import os
from tensorflow import keras
# from keras.callbacks import EarlyStopping
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt

In [2]:
%cd /Users/Kunal/Projects/TCH_CardiacSignals_F20/

/Users/kunal/Projects/TCH_CardiacSignals_F20


In [3]:
file_index = "1"
data = np.load(os.path.join("Working_Data", "Fixed_Dim_HBs_Idx" + file_index + ".npy"))

# lead_1_test = lead_1[30000:]
# look through all leads and degrees from 1 to 10 and do that stuff for all and save it out

# Use the entire lead train an autoencoder for each lead and then run the encoder on the leads train data

In [14]:
encoding_dim = 10
all_leads_reconstructed = np.empty([len(data),100,4])
all_leads_encoded = np.empty([len(data), encoding_dim, 4])
for num in range(4):
    current_lead = data[:,:,num]

    input_hb = keras.Input(shape = (100,))
    # es = keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

    encoded = keras.layers.Dense(encoding_dim, activation = 'tanh')(input_hb)
    decoded = keras.layers.Dense(100, activation = "tanh")(encoded)

    # maps input to reconstruction
    autoencoder = keras.Model(input_hb, decoded)

    encoder = keras.Model(input_hb, encoded)

    autoencoder.compile(optimizer='adam', loss = 'mse')

    for i in range(len(current_lead)):
        current_lead[i,:] = minmax_scale(current_lead[i,:], feature_range=(0,1), axis=0, copy=True)

    # callbacks=[es]
    autoencoder.fit(current_lead, current_lead, epochs = 50, shuffle=False)

    reconstruction = autoencoder.predict(current_lead)
    encode = encoder.predict(current_lead)

    all_leads_reconstructed[:,:,num] = reconstruction
    all_leads_encoded[:,:,num] = encode
    keras.backend.clear_session()

    reconstruction_save = os.path.join("Working_Data", "reconstructed_ae_" + str(encoding_dim) + "d_Idx" + str(file_index) + ".npy")
    encoded_save = os.path.join("Working_Data", "reduced_vae_" + str(encoding_dim) + "d_Idx" + str(file_index) + ".npy")
    np.save(reconstruction_save, all_leads_reconstructed)
    np.save(encoded_save,all_leads_encoded)


Epoch 1/50
1449/1449 [==============================] - 1s 723us/step - loss: 0.0091